In [1]:
# 케라스

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D, concatenate, Activation
from keras.models import Model

def fire(pre_layer, sq, ex):
    squeeze = Conv2D(sq, kernel_size=(1,1), padding='same')(pre_layer)
    squeeze = Activation('relu')(squeeze)
    expand_11 = Conv2D(ex, kernel_size=(1,1), padding='same')(squeeze)
    expand_11 = Activation('relu')(expand_11)
    expand_33 = Conv2D(ex, kernel_size=(3,3), padding='same')(squeeze)
    expand_33 = Activation('relu')(expand_33)
    x = concatenate([expand_11, expand_33])
    x = Activation('relu')(x)
    
    return x

inputs = Input(shape=(224, 224, 3))

x = Conv2D(96, kernel_size=(7,7), strides=2, padding='same')(inputs)
x = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)
x = fire(x, 16, 64)
x = fire(x, 16, 64)
x = fire(x, 32, 128)
x = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)
x = fire(x, 32, 128)
x = fire(x, 48, 192)
x = fire(x, 48, 192)
x = fire(x, 64, 256)
x = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)
x = fire(x, 64, 256)
x = Dropout((0.5))(x)
x = Conv2D(1000, kernel_size=(1,1), padding='same')(x)
x = GlobalAveragePooling2D()(x) # AvgPool이라고 쓰여져 있지만 Dense 대신에 쓰였으므로 GlobalAvgPool 씀
x = Activation('softmax')(x) # 최근엔 classifier로 FC(Dense)대신 GAP 많이 쓴다 - 파라미터 수 줄이고 과적합 피함

outputs = x

model = Model(inputs, outputs)

model.summary()

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 96) 14208       input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 96)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 56, 56, 16)   1552        max_pooling2d_1[0][0]            
__________

In [4]:
import torch
import torch.nn as nn

class fire_module(nn.Module):
    def __init__(self, in_ch,  sq, ex):
        super(fire_module, self).__init__()
        layers = []
        layers += [nn.Conv2d(in_ch, sq, kernel_size=1),
                   nn.ReLU(True)]
        expand1 = []
        expand1 += [nn.Conv2d(sq, ex, kernel_size=1), # in_channel: sq, out_channel: ex
                    nn.ReLU(True)]
        expand2 = []
        expand2 += [nn.Conv2d(sq, ex, kernel_size=3, padding=1),
                    nn.ReLU(True)]

        self.layers = nn.Sequential(*layers)
        self.expand1 = nn.Sequential(*expand1)
        self.expand2 = nn.Sequential(*expand2)

    def forward(self, x):
        x = self.layers(x)
        ex1 = self.expand1(x)
        ex2 = self.expand2(x)
        return torch.cat((ex1, ex2), 1)

class SqeezeNet(nn.Module):
    def __init__(self):
        super(SqeezeNet, self).__init__()
        layers = []
        layers += [nn.Conv2d(in_channels=3, out_channels=96, kernel_size=7, stride=2, padding=3),
                   nn.MaxPool2d(3, 2, 1), # filter_size, stride, padding
                   fire_module(96, 16, 64),
                   fire_module(128, 16, 64),
                   fire_module(128, 32, 128),
                   nn.MaxPool2d(3, 2, 1),
                   fire_module(256, 32, 128),
                   fire_module(256, 48, 192),
                   fire_module(384, 48, 192),
                   fire_module(384, 64, 256),
                   nn.MaxPool2d(3, 2, 1),
                   fire_module(512, 64, 256),
                   nn.Dropout(0.5),
                   nn.Conv2d(512, 1000, 1),
                   nn.AdaptiveAvgPool2d((1,1)), # batch * 1000 * 1 * 1
                   View(-1)] # batch * 1000 * 1 * 1 는 우리가 보기에 1차원이지만 컴퓨터가 보기엔 4차원이므로 그걸 해결하기 위함
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

class View(nn.Module):

    def __init__(self, *shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(x.shape[0], *self.shape)

if __name__ == '__main__':
    from torchsummary import summary
    model = SqeezeNet()
    summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 112, 112]          14,208
         MaxPool2d-2           [-1, 96, 56, 56]               0
            Conv2d-3           [-1, 16, 56, 56]           1,552
              ReLU-4           [-1, 16, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           1,088
              ReLU-6           [-1, 64, 56, 56]               0
            Conv2d-7           [-1, 64, 56, 56]           9,280
              ReLU-8           [-1, 64, 56, 56]               0
       fire_module-9          [-1, 128, 56, 56]               0
           Conv2d-10           [-1, 16, 56, 56]           2,064
             ReLU-11           [-1, 16, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]           1,088
             ReLU-13           [-1, 64, 56, 56]               0
           Conv2d-14           [-1, 64,